## Description
This is a notebook for debugging the quantity in the amiral code for both the python and the IDL version. 

At the moment, the IDL version is working properly but we do need to check the defintion for the Fourier variables (regarding fft shift and fft transform). We need to make sure that we are not being an idiot ;) 

#### Make sure before you perform fft, you shift the frequency domain back into the quarant ...




# Section
<details>
  <summary>Click to expand!</summary>
  
  ## Heading
  1. [Generate a PSF](#Generate-a-PSF)
    * [Zero Padding](#Zero-Padding)
    * [Setting up the system](#Setting-up-the-system)
    * [Pupil Function](#Pupil-Function)
    
  2.[Minimisation](#Minimisation)
    
  3.[How to fix the differences?](#How-to-fix-the-differences?)
    
  3.[Doesnt match with the IDL output](#Doesnt-match)
  
</details>

In [ ]:
# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
#Change to your path
os.chdir("/Users/alau/Repo/amiral")
from amiral import instructment, utils, parameter, gradient
from scipy.optimize import minimize 
%matplotlib inline

In [ ]:
# Global variable
# Parameters to modify
SEED = -1234       # Seed for random numbers generation
FLUX = 5e8         # Object total intensity [e-]
READ_OUT_NOISE = 10. # CCD read-out noise standard-deviation [e-]
DATA_DIR = "/Users/alau/IDLWorkspace/Data/Vesta_OASIS/"
DATA_FITS = DATA_DIR + "2018-06-08T05_27_05.809.fits"
DIMENSION = 256 # Dimension of the image
LOOP = True
test_data_dir = "/Users/alau/Data/amiral_fits/"

In [ ]:
# Setup of the system
fits_img = fits.open(DATA_FITS)
fits_img.info() 

obj = fits_img[0].data
obj_resize = np.zeros((DIMENSION,DIMENSION))

cuta = DIMENSION//2-128
cutb = DIMENSION//2+128

obj_resize[cuta:cutb,cuta:cutb] = obj

# Calibrating the flux
obj_resize = obj_resize/np.sum(obj_resize)*FLUX

aosys = instructment.aoSystem(diameter = 7., occ_ratio = 0.1, no_acutuator = 30,resolution_rad = 3.5e-8, wavelength = 500*1e-9, dimension = DIMENSION)
fX, fY, freqnull = aosys.psd_frequency_array(DIMENSION, aosys.samp_factor[0])

pupil = aosys.get_pupil_plane()

otf_tel = aosys.pupil_to_otf_tel(pupil, absolute = True)
psf_tel = np.real(utils.ifft2D((otf_tel)))

param = {
    "r0": 0.2,                  
    "background": 1.5e-10 ,      
    "amplitude": 2.1,       
    "ax": 0.05,                            
    "ay": 0.05, 
    "theta": 0., 
    "beta": 1.5,
    "mu": 1., 
    "rho0": 1., 
    "p": 1.
}

aosys_dict = {
        'diameter': 7 , 
        'occ_ratio': 0.1 , 
        'no_acutuator' : 30, 
        'wavelength': 500, 
        'dimension': 256,
        'resolution_rad' : 3.5e-8
    }
param_key, param_input = utils.dict2array(param)

psd_ao = aosys.psd_residual_ao (guess = param_input)
psd_halo = aosys.psd_residual_halo(r0=param["r0"])

psd = psd_halo + psd_ao 

integral, SR = aosys.psd_integral(psd, r0=param["r0"])
otf_atmo = aosys.otf_atmo(psd)
otf_total = aosys.otf_total(otf_tel, otf_atmo)
psf_total = aosys.psfao(otf_total)


print("\nSum of the PSF (which should be excatly 1.)", np.sum(np.abs(psf_total)))
print("\nMax of the otf_atmo: (which should be excatly 1.)", np.max(otf_atmo))
print("\nMax of the otf_tel: (which should be excatly 1.)", np.max(otf_tel))
print("\nMax of the OTF (which should be excatly 1.)", np.max(otf_total))

In [ ]:
# Import the Data from IDL
fits_img = fits.open("/Users/alau/Data/amiral_fits/image_noise.fits")
fits_img.info() 

conv_image = fits_img[0].data
print(np.max(conv_image), np.min(conv_image))

print(aosys.samp_factor)

## AMIRAL guess

In [ ]:
amiral_guess = {
    "r0": 0.2,                  
    "background": 1.5e-10 ,      
    "amplitude": 2.1,       
    "ax": 0.05,                            
    "beta": 1.5
}

In [ ]:
psf_param, psf_guess = utils.dict2array(amiral_guess)

amiralparam = parameter.amiralParam(img=conv_image, guess = psf_guess, aosys = aosys)

hyper_min, hyper_max = amiralparam.hyperparam_bound(psf_guess, p_upperbound = 100., debug=True)

hyper_guess = amiralparam.hyperparam_initial(psf_guess, debug = True)

param_min = np.asarray([0.01,0,0,1e-8,1.01])
param_max =  np.asarray([1.,1e8,1e8,1e3,10])

upperbound = np.concatenate((param_max, hyper_max))
lowerbound = np.concatenate((param_min, hyper_min))

param_numerical_condition = np.array([1., 1e-4, 1., 1., 1.])
#hyperparam_numerical_condition = np.array([hyper_guess[0], hyper_guess[1], 1.])
hyperparam_numerical_condition = np.array([hyper_guess[0], 2., 1.])

numerical_condition = np.concatenate((param_numerical_condition, hyperparam_numerical_condition))


# Note that I tried Boolean mask but it doesnt work ...
param_mask = np.array([1, 1, 1, 1, 1])
hyper_param_mask = np.array([1, 1, 1])

mask = np.concatenate((param_mask,hyper_param_mask))

psf_guess = np.concatenate((psf_guess, hyper_guess))

amiral_child = parameter.amiral(img=conv_image, guess=psf_guess, aosys = aosys, upperbound = upperbound, lowerbound= lowerbound, numerical_condition = numerical_condition, fourier_variable = amiralparam.fourier_variable, mask = mask)

amiral_child.varible4criterion(psf_guess)
print(psf_guess)

## Minimisation

In [ ]:
print(psf_guess)

In [ ]:
print(conv_image.shape)

## Debug

#### Matched with IDL output
- fourier_variable[ft(im)]
- image
- fourier_variable["ft(mean_object)"]

In [ ]:
# image (Matched IDL definition as it is directly copied from IDL)
print("fourier_variable[ft(im)]\n=============================")
utils.info(conv_image)

In [ ]:
# ft_im - matched the IDL output
ft_im = utils.fft2D(conv_image, DIMENSION)
ft_image = np.fft.fft2(conv_image)

plt.imshow(np.real(np.log10(ft_image)))

print("fourier_variable[ft(im)]\n=============================")
utils.info(ft_im, imaginary = True)

print("fourier_variable[absft(im)]\n=============================")
utils.info(np.abs(ft_image), imaginary = True)

In [ ]:
# fourier_variable["ft(mean_object)"] - matched the IDL output
utils.info(amiral_child.fourier_variable["ft(mean_object)"], imaginary = True)
plt.imshow(np.log10(np.abs(amiral_child.fourier_variable["ft(mean_object)"]+0.1)))

#### How to fix the differences? 
From directly taking the PSD ouptut from IDL and using our functions to produce the OTF, we know the errors that we have been getting **is not from Fourier Transform**. It is likely due to different defintions in the PSD:

Therefore, we shall check the definition of the PSDs and compare the output!

**Potential reason**
- Look at the PSD_AO from IDL, at the edge of the cutoff frequency, there are a few spots ... 

In [ ]:
# psd_halo - matched the IDL output 
fXY = aosys.fX**2 + aosys.fY**2
print(np.where (fXY == 0))
fits_img = fits.open(test_data_dir +"psd_halo.fits")
psd_halo_IDL = fits_img[0].data
diff = psd_halo_IDL - psd_halo

diff = utils.view_diff(psd_halo_IDL,psd_halo)

utils.info(diff)

ind = np.where (diff == np.min(diff))

plt.imshow(diff[256-80:256+80,256-80:256+80])

diff[np.where((diff == np.min(diff)))] = 1e-10

plt.imshow(diff[256-80:256+80,256-80:256+80])

In [ ]:
fits_img = fits.open(test_data_dir +"psd_halo_array.fits")
fXY_IDL = fits_img[0].data
fXY_array = (fXY >= aosys.ao_cutoff_freq**2)

diff = utils.view_diff(fXY_array,fXY_IDL)


utils.info(diff)

In [ ]:
# test_psd_halo = 0.0229 * (0.15**(-5./3.))* (((1./10.**2)+fXY)**(-11./6.))
Lext = 10.

test_psd_halo = ((1./Lext**2)+fXY)**(-11./6.)

fits_img = fits.open(test_data_dir +"psd_halo_test.fits")
test_psd_halo_IDL = np.transpose(fits_img[0].data)


diff = utils.view_diff(test_psd_halo_IDL,test_psd_halo)

utils.info(diff)

plt.imshow(diff[256-80:256+80,256-80:256+80])

In [ ]:
# fig, ax =  plt.subplots(1,3)

# pos = ax[0].imshow(diff)
# ax[0].set_title('PSD halo diff')
# fig.colorbar(pos, ax=ax[0])

# pos = ax[1].imshow(psd_halo)
# ax[1].set_title('PSD halo (python)')
# fig.colorbar(pos, ax=ax[1])


# pos = ax[2].imshow(psd_halo_IDL)
# ax[2].set_title('PSD halo (IDL)')
# fig.colorbar(pos, ax=ax[2])

# plt.tight_layout()

In [ ]:
# psd - matched the IDL output 
fits_img = fits.open(test_data_dir+"psd_total.fits")
psd_IDL= fits_img[0].data

psd_ao_IDL = psd_IDL-psd_halo_IDL

psd_ao_diff = psd_ao - psd_ao_IDL 

diff = utils.view_diff(psd_ao, psd_ao_IDL)

utils.info(psd_ao_diff)

In [ ]:
# fig, ax =  plt.subplots(1,3)

# pos = ax[0].imshow(psd_ao_diff[256-70:256+70,256-70:256+70])
# ax[0].set_title('PSD AO Diff')
# fig.colorbar(pos, ax=ax[0])

# pos = ax[1].imshow(np.log10(psd_ao[256-70:256+70,256-70:256+70]))
# ax[1].set_title('PSD AO (python)')
# fig.colorbar(pos, ax=ax[1])


# pos = ax[2].imshow(np.log10(psd_ao_IDL[256-70:256+70,256-70:256+70]))
# ax[2].set_title('PSD AO (IDL)')
# fig.colorbar(pos, ax=ax[2])

# plt.tight_layout()

In [ ]:
# plt.imshow(np.log10(diff[256-80:256+80,256-80:256+80]))

In [ ]:
# fig, ax =  plt.subplots(1,3)

# pos = ax[0].imshow(diff)
# ax[0].set_title('PSD diff')
# fig.colorbar(pos, ax=ax[0])

# pos = ax[1].imshow(np.log10(psd[256-80:256+80,256-80:256+80]))
# ax[1].set_title('PSD (python)')
# fig.colorbar(pos, ax=ax[1])


# pos = ax[2].imshow(np.log10(psd_IDL[256-80:256+80,256-80:256+80]))
# ax[2].set_title('PSD (IDL)')
# fig.colorbar(pos, ax=ax[2])

# plt.tight_layout()

In [ ]:
# # otf_atmo - matched the IDL output
# #utils.info(otf_atmo, imaginary=True)
# fits_img = fits.open("/Users/alau/otf_atmo_abs.fits")
# otf_atmo_IDL = fits_img[0].data
# diff = np.abs(otf_atmo_IDL - otf_atmo)

# fits_img = fits.open("/Users/alau/Bphi.fits")
# Bphi_IDL = fits_img[0].data

In [ ]:
# # how you get otf_atmo 
# L = aosys.diameter * aosys.sampling # Checked
# B_phi_IDL = np.fft.fft2(np.fft.fftshift(psd_IDL)) / L**2 # Checked
# D_phi = np.fft.fftshift(np.real(2.* (B_phi_IDL.max() - B_phi_IDL))) # Checked
# otf_atmo_psd_IDL = np.exp(-D_phi/2)

# print(type(L))
# fig, ax = plt.subplots()
# pos = ax.imshow(np.real(Bphi_IDL-B_phi_IDL))
# ax.set_title("Numerical noise of FFT")
# # pos = ax.imshow(np.real(diff))
# fig.colorbar(pos)

#### OTF(atmosphere)
From the above figure, we find out that the numerical noise shoud be 1e-16. However, we are seeing 1e-8 difference between the IDL and the python output. 

**Is there any other sources of errros?**

In [ ]:
# fig, ax = plt.subplots()
# #pos = ax.imshow(np.real(otf_atmo_IDL-otf_atmo_psd_IDL))
# pos = ax.imshow(otf_atmo_IDL-otf_atmo)
# fig.colorbar(pos)

In [ ]:
# fig, ax =  plt.subplots(1,3)

# print(diff.dtype)
# print(otf_atmo[0,0])

# pos = ax[0].imshow(np.log10(diff[256-80:256+80,256-80:256+80]))
# ax[0].set_title('OTF_atmo diff')
# fig.colorbar(pos, ax=ax[0])

# pos = ax[1].imshow(otf_atmo[256-80:256+80,256-80:256+80])
# ax[1].set_title('OTF_atmo (python)')
# fig.colorbar(pos, ax=ax[1])


# pos = ax[2].imshow(otf_atmo_IDL[256-80:256+80,256-80:256+80])
# ax[2].set_title('OTF_atmo (IDL)')
# fig.colorbar(pos, ax=ax[2])

# plt.tight_layout()

#### In progress

For each variable, check the following: 
- max, min, mean, sum 
- sum(abs())


One thing 
- we know the otf_atmo is correct but the otf_tel is not correct due to the different definition. 

Question 

- We are using || for fourier variables, as long as the sum is the same, it should be fine? 

Current problem 

- I dont think we have the same defintion of fft and ifft, in order the have the same output, we must match all the defintions ... 

#### Diffraction-limited OTF 
There is still something wrong with the defintion of the diffraction-limited OTF.

Due to the error in otf_tel, the ft_psf is off too ... which means terms containing ft_psf is wrong ... 

Let's investigate!

Therefore, from the python version, we can see that the difference between the real and the absolute value should not matter that much if phase is not playing a huge role. 

However, the defition of the OTF shows that OTF is a complex function.

In [ ]:
# otf_total_test = amiral_child.update_otf(psf_guess)

# plt.imshow(np.real(otf_total_test))

In [ ]:
# tfh = utils.load_fits("/Users/alau/tfh_real.fits")
# diff = amiral_child.fourier_variable["ft(psf)"] - tfh

# fig, ax =  plt.subplots()
# pos = ax.imshow(np.real(diff))
# ax.set_title('OTF total diff')
# fig.colorbar(pos, ax=ax)

# utils.info(diff)

# print(amiral_child.fourier_variable["ft(psf)"][256,256])
# print(tfh[256,256])

In [ ]:
# # Check the amiral_child.fourier_variable["ft(image)"]
# ft_obj = np.fft.fft2(conv_image)
# ft_image = amiral_child.fourier_variable["ft(image)"]

# tfi_real = utils.load_fits("/Users/alau/tfi_real.fits")

# diff = utils.view_diff(tfi_real,ft_image)
# utils.info(diff)

# print(diff[256,256])

# print(ft_image[256,256])

In [ ]:
# ft_mean_obj = amiral_child.fourier_variable["ft(mean_object)"]
# ft_o_moy = np.fft.fftshift(utils.load_fits("/Users/alau/ft_o_moy_real.fits"))

# diff = utils.view_diff(ft_mean_obj,ft_o_moy)
# utils.info(ft_mean_obj-ft_o_moy)


# np.where(ft_mean_obj == np.max(ft_mean_obj))

In [ ]:
# tfi2 = np.fft.fftshift(np.transpose(utils.load_fits("/Users/alau/tfi2.fits")))
# diff = tfi2 - amiral_child.fourier_variable["error2"]
# utils.info(diff)

# fig, ax =  plt.subplots()
# pos = ax.imshow(diff[256-20:256+20,256-20:256+20])
# ax.set_title('error total diff')
# fig.colorbar(pos, ax=ax)

# np.where(diff == np.max(diff))

In [ ]:
# # fourier_variable["psd(psf)"] 
# utils.info(amiral_child.fourier_variable["psd(psf)"], imaginary = True)

In [ ]:
# tfh2 = np.fft.fftshift(utils.load_fits("/Users/alau/tfh2.fits"))
# diff = tfh2 - amiral_child.fourier_variable["psd(psf)"]
# utils.info(diff)

# fig, ax =  plt.subplots()
# pos = ax.imshow(diff[256-20:256+20,256-20:256+20])
# ax.set_title('PSD total diff')
# fig.colorbar(pos, ax=ax)

In [ ]:
# # mean_error (tfi2_moy)
# # Minimum is off but the rest matched the IDL ouput
# mean_error = utils.mean_cir_array(amiral_child.fourier_variable["error2"])
# utils.info(mean_error, imaginary = True)
# np.sum(mean_error)

In [ ]:
# # meam_ft_h2(tfh2_moy)
# # mean and the minimum are off (given that the otf is off too ...)
# meam_ft_h2 = utils.mean_cir_array(amiral_child.fourier_variable["psd(psf)"])
# utils.info(meam_ft_h2, imaginary = True)
# # sum is off
# np.sum(meam_ft_h2)

In [ ]:
# # sb_ini
# # matched the IDL output

# dimension = 512

# sb_ini = (np.sum(amiral_child.fourier_variable["error2"][0,0:dimension]) + 
#           np.sum(amiral_child.fourier_variable["error2"][dimension-1, 0:dimension]) 
#           + np.sum(amiral_child.fourier_variable["error2"][1:dimension-1, 0]) + 
#             np.sum(amiral_child.fourier_variable["error2"][1:dimension-1, dimension-1])) / (4.*dimension - 4.)

# print(sb_ini)

In [ ]:
# # All off - length is correct
# print("\nlen:", len(amiral_child.fourier_variable["error2"][0,0:dimension]))
# utils.info(amiral_child.fourier_variable["error2"][0,0:dimension])
# np.sum(amiral_child.fourier_variable["error2"][0,0:dimension]) - 1001557.7383473666

In [ ]:
# # All off
# print("\nlen:", len(amiral_child.fourier_variable["error2"][dimension-1, 0:dimension]))
# utils.info(amiral_child.fourier_variable["error2"][dimension-1, 0:dimension])
# np.sum(amiral_child.fourier_variable["error2"][dimension-1, 0:dimension]) - 1002877.2089995614

In [ ]:
# print("\nlen:", len(amiral_child.fourier_variable["error2"][1:dimension-1, 0]))
# utils.info(amiral_child.fourier_variable["error2"][1:dimension-1, 0])
# np.sum(amiral_child.fourier_variable["error2"][1:dimension-1, 0]) - 1089953.2424355166

In [ ]:
# print("\nlen:", len(amiral_child.fourier_variable["error2"][1:dimension-1, dimension-1]))
# utils.info(amiral_child.fourier_variable["error2"][1:dimension-1, dimension-1])
# np.sum(amiral_child.fourier_variable["error2"][1:dimension-1, dimension-1]) - 1086312.6571350598

In [ ]:
# # k_ini
# # almost matched IDL output (1.1213083933789605e+17)
# k_ini = mean_error[1] 
# k_ini - 1.1213083933789605e+17

In [ ]:
# # mu_ini 
# # matched IDL output
# mu_ini = sb_ini / k_ini
# print(mu_ini)
# mu_ini - 1.8240768346243759e-14

Minimisation - Recheck!

In [ ]:
# # amiral_child.fourier_variable["rho"]
# # matched the IDL output
# rho_IDL = np.fft.fftshift(utils.load_fits("/Users/alau/rho.fits"))
# diff = utils.view_diff(rho_IDL, amiral_child.fourier_variable["rho"])

In [ ]:
# # fourier_variable["good_rho"] - checked
# arr = amiral_child.fourier_variable["good_rho"]
# utils.info(arr)
# np.sum(arr)

In [ ]:
# # checking the initialisation - matched IDL ouput
# amiral_child.hyperparam_initial(psf_guess)

In [ ]:
# # fourier_variable["psd_object_ini"] - dspo
# # - matched IDL output
# # 1.686e-07   1.000e+00   1.152e-04     3.021e+01
# dspo = np.fft.fftshift(np.transpose(utils.load_fits("/Users/alau/dspo.fits")))
# diff = utils.view_diff(dspo, amiral_child.fourier_variable["psd_object_ini"])

# utils.info(diff)

In [ ]:
# # fourier_variable["psd_model_i"] - denom - Note: The error is coming from dspo and mu
# # matched the IDL output
# # 1.824e-14   1.000e+00   4.300e-05 1.127e+01

# denom = np.fft.fftshift(np.transpose(utils.load_fits("/Users/alau/denom.fits")))
# denom_python = amiral_child.fourier_variable["psd(psf)"]* amiral_child.fourier_variable["psd_object_ini"] + psf_guess[7]
# diff = utils.view_diff(denom, amiral_child.fourier_variable["psd_model_i"])

# utils.info(diff)

In [ ]:
# # fourier_variable["k_hat"] - does not match - continue in here!
# print(amiral_child.fourier_variable["k_hat"] -1.3093721312891771e+17)

# tfi2_swap = np.transpose(tfi2)
# denom_swap = np.transpose(denom)

# # Try to use the quantity from IDL to check the eqaution
# k_hat = 1/(dimension**2) *np.sum ((tfi2_swap/denom_swap)[amiral_child.fourier_variable["good_rho"]])

# print(k_hat/1.3093721312891771e+17)

# # From the result, I 

In [ ]:
# # k_hat has been fixed
# test_k = amiral_child.fourier_variable["error2"] / amiral_child.fourier_variable["psd_model_i"]

# utils.info(amiral_child.fourier_variable["error2"])
# print(np.sum(arr))

# test_sum = np.sum(test_k[amiral_child.fourier_variable["good_rho"]])
# print(test_sum)

In [ ]:
# # fourier_variable["psd_object_ini"] - dspo
# # doesnt not match ... 
# # 1.651e+10   9.791e+16   1.128e+13    2.958e+18
# dspo_IDL =  np.transpose(np.fft.fftshift(utils.load_fits("/Users/alau/dspo.fits")))
# diff = utils.view_diff(dspo_IDL,amiral_child.fourier_variable["psd_object_ini"])
# utils.info(diff)

In [ ]:
# # fourier_variable["psd_image_est"] - dsp_i
# # doesnt match ... 
# # 1.786e+03   9.791e+16   4.210e+12   1.104e+18
# arr = amiral_child.fourier_variable["psd_image_est"]
# utils.info(arr)
# print(np.sum(arr))

In [ ]:
# dsp_ratio_IDL = np.fft.fftshift(utils.load_fits("/Users/alau/dsp_ratio.fits"))
# dsp_ratio = amiral_child.fourier_variable["psd_object"] / amiral_child.fourier_variable["psd_image_est"]

# plt.imshow(dsp_ratio_IDL/dsp_ratio)

# utils.info(dsp_ratio_IDL/dsp_ratio)

In [ ]:
# # Fourier variable dsp_bruit - checked
# dsp_bruit = amiral_child.fourier_variable["psd_noise"]
# print(dsp_bruit)



In [ ]:
# # dsp_objet 
# dsp_objet_IDL = np.transpose(np.fft.fftshift(utils.load_fits("/Users/alau/dsp_objet.fits")))
# dsp_objet = amiral_child.fourier_variable["psd_object"]

# diff = utils.view_diff(dsp_objet,dsp_objet_IDL)
# utils.info(dsp_objet/dsp_objet_IDL)

In [ ]:
# # psd_image_est
# # dsp_objet 
# dsp_i_IDL = np.transpose(np.fft.fftshift(utils.load_fits("/Users/alau/dsp_i.fits")))
# dsp_i = amiral_child.fourier_variable["psd_image_est"]

# diff = utils.view_diff(dsp_i,dsp_i_IDL)
# utils.info(dsp_i/dsp_i_IDL)


# utils.info(dsp_i_IDL)

# IDL 
grad_psf: [313637.36820290622      -422776.16167403548      -20852.509443358213      -338842.43443541240       22386.038647048441]

grad_hyper: [-1.5547975389990738e+18       42539.024177912448      -111031.55773929584]

#### python
grad_psf:
[ 3.76661374e+05  1.76712368e-06  1.05124255e-04 -7.47756685e-03
  3.44429096e-04  0.00000000e+00  0.00000000e+00]
  
  
grad_hyper:
[5.82125344e+18
  3.72709536e+04 -9.05318129e+04]

## What to do now? 

1. Check the gradient function in python when bringing the analytical grad to the minimiser. Current gradient seems to be too large!

2. One thing to check is the defintion of the gradient 
    - Are they centred at the centre of the image or at the corner? 
    - Subtracting the wrong version will lead to an error

3. 


In [ ]:
aosys = instructment.aoSystem(diameter = aosys_dict['diameter'], occ_ratio = aosys_dict['occ_ratio'], 
        no_acutuator= aosys_dict['no_acutuator'], wavelength = aosys_dict['wavelength']*1e-9, resolution_rad = aosys_dict['resolution_rad'], 
        dimension=aosys_dict['dimension'])

In [ ]:
# psf_dict = {
#         'r0' : 0.15, 
#         'background': 1e-10 , 
#         'amplitude': 1.8, 
#         'ax' : 0.05, 
#         'beta' : 1.5,
#     }

# psf_keys, psf_guess = utils.dict2array(psf_dict)
print(psf_guess)

In [ ]:
test_grad = aosys.gradient(psf_guess, debug = True)
PSD_grad_IDL = np.zeros((len(psf_guess), aosys.N_padded[0],aosys.N_padded[0]))
diff = np.zeros((len(psf_guess), aosys.N_padded[0],aosys.N_padded[0]))
for i in range (len(psf_guess)-3): 
    PSD_grad_IDL [i, :, :] = utils.load_fits(test_data_dir+"PSD_grad_"+str(i)+".fits")
    diff [i, :, :] = utils.view_diff(test_grad[i], PSD_grad_IDL[i])

In [ ]:
print("Minimum: ",np.where(diff == np.min(diff)))

for i in range (len(psf_guess)-3):
    print(i)
    print("Sum of diff grad: %f" %(np.sum(test_grad[i,:,:]) - np.sum(PSD_grad_IDL[i,:,:])))
    print('PSD_grad_IDL centre %i : %f' %(i, PSD_grad_IDL[i, aosys.freqnull[0], aosys.freqnull[1]]))
    print('PSD_grad_python centre %i : %f' %(i, test_grad[i, aosys.freqnull[0], aosys.freqnull[1]]))
    print("Max is %f \n%s" %(np.max(diff[i]),np.where(diff[i] == np.max(diff[i]))))
    print("Minimum is %f \n%s"%(np.min(diff[i]),np.where(diff[i] == np.min(diff[i]))))


In [ ]:
# test_grad = aosys.gradient(psf_guess, debug = False)*(-amiral_child.fourier_variable["ft()"])
grad_IDL = np.zeros((len(psf_guess), aosys.N_padded[0],aosys.N_padded[0]))
OTF_grad = aosys.gradient(psf_guess)*(-amiral_child.fourier_variable["ft(psf)"])
test_grad = PSD_grad_IDL*(-amiral_child.fourier_variable["ft(psf)"])

for i in range (len(psf_guess)-3):
    grad_IDL [i, :, :] = np.transpose(utils.load_fits(test_data_dir+"OTF_grad_"+str(i)+".fits"))
    diff = utils.view_diff(grad_IDL[i], OTF_grad[i])
    utils.info(diff)
    print(np.where(diff == diff.max()))
    
    


In [ ]:
diff = utils.view_diff(test_grad[0], grad_IDL[0])

utils.info(diff)
print(np.where(diff == diff.max()))

In [ ]:
plt.imshow(grad_IDL[4])

In [ ]:
# grad[0] = (fXY >=(self.ao_cutoff_freq**2.)) * (-5./3.)* 0.0229 * r0 **((-8./3.)) * ((1./Lext**2.)+fXY)**((-11./6.)) # grad_r0

r0 = 0.15
fX, fY, freq_null = aosys.psd_frequency_array(aosys_dict['dimension'], aosys.samp_factor[0])
Lext = 10. 

fXY = fX**2 + fY**2
plt.imshow(fXY)

grad0 = (fXY >=(aosys.ao_cutoff_freq**2.)) * (-5./3.)* 0.0229 * r0 **((-8./3.)) * ((1./Lext**2.)+fXY)**((-11./6.)) # grad_r0



In [ ]:
plt.imshow(grad0)

In [ ]:
plt.imshow(aosys.fX**2 + aosys.fY**2)

In [ ]:
error2_IDL = np.fft.fftshift(utils.load_fits(test_data_dir+"error2.fits"))
error2_python = amiral_child.fourier_variable["error2"]


In [ ]:
diff = utils.view_diff (error2_IDL,error2_python)
utils.info(diff)

plt.imshow(diff[256-80:256+80,256-80:256+80])


a = np.where (np.max(diff) == diff)

print(diff[a],a)
#plt.imshow(np.fft.fftshift(np.log10(error2_IDL-error2_python)))

In [ ]:
img_IDL = utils.load_fits(test_data_dir+"image_noise.fits")
utils.info(img_IDL)

In [ ]:
plt.imshow(np.real(np.log10(error2_IDL)))

In [ ]:
ft_image_IDL = utils.load_fits(test_data_dir+"ft_im.fits")

utils.info(ft_image_IDL)

plt.imshow(np.log10(ft_image_IDL))

In [ ]:
ft_image = np.fft.fftshift(np.fft.fft2(conv_image))
utils.info(ft_image)
plt.imshow(np.real(np.log10(ft_image)))

In [ ]:
diff = utils.view_diff(ft_image,ft_image_IDL)
utils.info(diff)


In [ ]:
ft_img = np.fft.fftshift(np.fft.fft2(conv_image))

diff = utils.view_diff(np.real(ft_img),amiral_child.fourier_variable["ft(image)"])

utils.info(diff)

In [ ]:
rho = amiral_child.fourier_variable["rho"]
plt.imshow(rho)



test = utils.dist(512)
utils.info(test)

In [ ]:
dimension = 512


sb_ini = (np.sum(amiral_child.fourier_variable["error2"][0,0:dimension-1]) + 
    np.sum(amiral_child.fourier_variable["error2"][dimension-1, 0:dimension-1]) + np.sum(amiral_child.fourier_variable["error2"][1:dimension-1, 0]) + 
    np.sum(amiral_child.fourier_variable["error2"][1:dimension-1, dimension-1])) / (4.*dimension - 4.)

mean_error = utils.mean_cir_array(amiral_child.fourier_variable["error2"])


k_ini = mean_error[1]

In [ ]:
k_ini

In [ ]:
sb_ini

In [ ]:
amiral_child.fourier_variable['k_hat']/1.0467463006917365e17

In [ ]:
psd_model_i_IDL = np.fft.fftshift(utils.load_fits(test_data_dir+"psd_model_i.fits"))

In [ ]:
dJdo = .5 * (amiral_child.fourier_variable["psd(psf)"]/amiral_child.fourier_variable["psd_model_i"]-
        (amiral_child.fourier_variable["psd(psf)"]*amiral_child.fourier_variable["error2"])/
        (amiral_child.fourier_variable["k_hat"]*(amiral_child.fourier_variable["psd_model_i"]**2)))
dJdo_IDL = utils.load_fits(test_data_dir+"dJdo.fits")

In [ ]:
denom = utils.load_fits(test_data_dir+"psd_model_i.fits")
tfh2 = utils.load_fits(test_data_dir+"psd_psf.fits")
tfi2 = utils.load_fits(test_data_dir+"error2.fits")
k_hat = 1.0187311980095106e+17


In [ ]:
dJdo_test = 0.5 * (tfh2/denom-(tfh2*tfi2)/(k_hat*(denom**2)))
#.5D * (tfh2/denom-(tfh2*tfi2)/(k_chap*denom^2))


diff = utils.view_diff(dJdo,np.fft.fftshift(dJdo_test))


utils.info(diff)

plt.imshow(dJdo)

In [ ]:
plt.imshow(np.fft.fftshift(dJdo_IDL))


utils.info(dJdo_IDL-dJdo)

In [ ]:
test = tfh2/denom
test_python = amiral_child.fourier_variable["psd(psf)"]/amiral_child.fourier_variable["psd_model_i"]


# test = tfh2
# test_python = amiral_child.fourier_variable["psd(psf)"]




utils.info(np.fft.fftshift(test) - test_python)

In [ ]:
diff = utils.view_diff(dJdo, dJdo_IDL)

utils.info(diff)

In [ ]:
fig, ax1 =  plt.subplots()
pos = ax1.imshow(dJdo, cmap='Reds', interpolation='none')
fig.colorbar(pos, ax=ax1)


utils.info(dJdo)

In [ ]:
fig, ax1 =  plt.subplots()
pos = ax1.imshow(np.log10(np.transpose(dJdo_IDL)/dJdo), cmap='Reds', interpolation='none')
fig.colorbar(pos, ax=ax1)


utils.info(dJdo_IDL)

In [ ]:
for i in range (len(psf_guess)-3): 
        grad_IDL [i, :, :] = np.transpose(utils.load_fits(test_data_dir+"OTF_grad_"+str(i)+".fits"))

In [ ]:
plt.imshow(np.log10(grad_IDL[0]))

In [ ]:
otf_IDL = np.fft.fftshift(utils.load_fits(test_data_dir+ "otf_real.fits"))

In [ ]:
psf_guess[-2] = 2.0
psf_guess[-1] = 3.0

print(psf_guess)

In [ ]:
amiral_child.varible4criterion(psf_guess)
grad_python = amiral_child.gradient(psf_guess, debug = True)

In [ ]:
grad_IDL = np.array([
        112666.46975593880,
        -556963.27478424134,      
        -19232.183893144967,     
        -562320.62551296421,
        47246.817020119110,
        -4809744466462.9717,
        29179.449077962287,
        -71791.829213569916])

# potentially fixed! See the output from pytest!

In [ ]:
grad_error = (grad_python-grad_IDL)
# with a fixed k_hat

In [ ]:
print((grad_error/grad_IDL)*100)

In [ ]:
amiral_child.gradient(psf_guess)

In [ ]:
print(help(amiral_child))

In [ ]:
test = np.array([0.05,0.01,0.0,0.025,1.5,2.5564283363852407e-09,2.0,3.0])




amiral_child.marg_criterion(psf_guess, debug = True)